 <div class="alert alert-block alert-info">
    <b>Note:</b> 
    Interchange is in the process of replacing ParmEd in many workflows, but it still in an alpha testing phase. Our internal tests indicate it is reliable for many small-molecule systems, but it is not yet reliable for complex, multi-component systems and there are likely still rough edges throughout. Feedback is welcome on the <a href=https://github.com/openforcefield/openff-interchange/issues/>Interchange issue tracker.</a></div>

## Using OpenFF force fields in Amber and GROMACS

The Open Forcefield Toolkit can create parametrized `openmm.System` objects that can be natively simulated with OpenMM. This example shows the Interchange project can enable parallel workflows using Amber and GROMACS.

### Preparing an OpenFF Topology

We start by loading a PDB file containing one copy of ethanol and cyclohexane. Our goal is to create an OpenFF `Topology` object describing this system that we can parametrize with the SMIRNOFF-format "Sage" force field.

The two `Molecule` objects created from the SMILES strings can contain information such as partial charges and stereochemistry that is not included in an OpenMM topology. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the "Sage" force field. Note that the OpenFF Toolkit produces partial charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import pdb

In [2]:
hexa_pmp = Molecule.from_file("../../../build_polymer/pmp/pmp_hexamer_mbuild.mol")

In [3]:
# Load the PDB file using OpenMM and save the OpenMM Topology
pdbfile = app.PDBFile("../../../build_polymer/pmp/pmp_hexamer_mbuild_renum.pdb")
omm_topology = pdbfile.topology
omm_topology

<Topology; 1 chains, 8 residues, 285 atoms, 302 bonds>

In [4]:
# Create the OpenFF Topology.
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[hexa_pmp]
)


In [5]:
# Modified OpenFF to increase maxAtoms for AM1BCC method to 500 atoms
if not os.path.exists('terphenyl_pmp_hexamer_charges.sdf'):
    hexa_pmp.assign_partial_charges(partial_charge_method='am1bcc')
    hexa_pmp.to_file('terphenyl_pmp_hexamer_charges.sdf', file_format='sdf')
else:
    hexa_pmp = Molecule.from_file('terphenyl_pmp_hexamer_charges.sdf')

/home/tfobe/Software/openff-toolkit/openff/toolkit/topology/molecule.py:2404: UserWarning: compute_partial_charges_am1bcc is deprecated and will be removed in version 0.12.0. Use assign_partial_charges(partial_charge_method='am1bcc') instead.
  warnings.warn(
/home/tfobe/Software/openff-toolkit/openff/toolkit/topology/molecule.py:2504: UserWarning: Warning! Partial charge method 'am1bcc' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 285 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  warnings.warn(


### Preparing an OpenFF ForceField

Once the `ForceField` class is imported, the only decision to make is which force field to use. An exhaustive list of force fields released by the Open Force Field Initiative can be found [here](from openff.toolkit.typing.engines.smirnoff import ForceField
).

Here we will use force field from the "Sage" line.

In [6]:
forcefield = ForceField("openff-2.0.0.offxml")

### Preparing an OpenMM System

Once a force field and topology have been loaded, an `openmm.System` can be generated natively with the OpenFF Toolkit.

In [7]:
omm_system = forcefield.create_openmm_system(off_topology, charge_from_molecules = [hexa_pmp])
omm_system

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7f656b64f6c0> >

### Preparing an Interchange object

To exports to engines other than OpenMM, we will make use of the [Interchange](https://openff-interchange.readthedocs.io/) project. There is a high-level `Interchange.from_smirnoff` function that consumes OpenFF Toolkit and ForceField objects and produces an `Interchange` object which can then be exported to formats understood by other molecular simulation engines. This extra step is needed to provide a clean interface between _applied_ parameters and engines. Note also that this step does not require an OpenMM System to be generated; `ForceField.create_openmm_system` does not need to be called to use Amber and GROMACS.

In [8]:
from openff.interchange.components.interchange import Interchange

interchange = Interchange.from_smirnoff(
    force_field=forcefield,
    topology=off_topology,
)
interchange.positions = pdbfile.positions
interchange

/home/tfobe/Software/openff-toolkit/openff/toolkit/topology/molecule.py:2504: UserWarning: Warning! Partial charge method 'am1bccelf10' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 285 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  warnings.warn(


Interchange with 7 potential handlers, non-periodic topology with 285 atoms.

### Exporting to Amber and GROMACS files

Once an `Interchange` object has been constructed, its API can be used to export to files understood by GROMACS, Amber, and more.

In [9]:
# Export GROMACS files.
interchange.to_top("terphenyl_pmp_hexamer.top")
interchange.to_gro("terphenyl_pmp_hexamer.gro")

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/openff/interchange/interop/internal/gromacs.py:164: UserWarning: WARNING: System defined with no box vectors, which GROMACS does not offically support in versions 2020 or newer (see https://gitlab.com/gromacs/gromacs/-/issues/3526). Setting box vectors to a 5  nm cube.
  warnings.warn(


### Validating the conversion to Amber files

The Interchange project includes functions that take in an `Interchange` object and call out to simulation engines to run single-point energy calculations (with no minimization or dynamics) for the purpose of validating the export layer with each engine. Under the hood, each of these functions calls API points like those used above while converting to files understood by each engine. These rely on having each engine installed and accessible in the current `$PATH`.

In [10]:
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies

In [11]:
openmm_energies = get_openmm_energies(interchange)
openmm_energies.energies

{'Bond': 311.73142241702254 <Unit('kilojoule / mole')>,
 'Angle': 1866.5743065145732 <Unit('kilojoule / mole')>,
 'Torsion': 604.7721035865441 <Unit('kilojoule / mole')>,
 'vdW': 30129.28039207037 <Unit('kilojoule / mole')>,
 'Electrostatics': -1296.6424149706318 <Unit('kilojoule / mole')>}

In [12]:
!cat terphenyl_pmp_hexamer.top
!cat terphenyl_pmp_hexamer.gro
amber_energies = get_gromacs_energies(interchange)
amber_energies.energies

; Generated by Interchange
[ defaults ]
; nbfunc	comb-rule	gen-pairs	fudgeLJ	fudgeQQ
     1	     2	no     0.500000 0.833333

[ atomtypes ]
;type, bondingtype, mass, charge, ptype, sigma, epsilon
C1               6 12.01078 0.0000000000000000 A     0.337953176162662 0.4553891161106184
O1               8 15.99943 0.0000000000000000 A     0.3025106490435313 0.7048581468486769
C2               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C3               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C4               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C5               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C6               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C7               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635030558377792
C8               6 12.01078 0.0000000000000000 A     0.3480646886945065 0.3635

Generated by Interchange
285
    1HEX     C1    1   0.21800000   0.49770000  -0.08130000
    1HEX     O1    2   0.10290000   0.41490000  -0.09520000
    1HEX     C2    3   0.09800000   0.27920000  -0.05990000
    1HEX     C3    4   0.20870000   0.21120000  -0.00610000
    1HEX     C4    5   0.19960000   0.07520000   0.02790000
    1HEX     C5    6   0.31680000   0.00390000   0.08500000
    1HEX     C6    7   0.44570000   0.02520000   0.03300000
    1HEX     C7    8   0.55590000  -0.04200000   0.08670000
    1HEX     C8    9   0.53930000  -0.13160000   0.19350000
    1HEX     C9   10   0.65440000  -0.20330000   0.25150000
    1HEX     O2   11   0.77090000  -0.18620000   0.20680000
    1HEX    C10   12   0.41090000  -0.15300000   0.24560000
    1HEX    C11   13   0.30060000  -0.08600000   0.19200000
    1HEX    C12   14   0.07820000   0.00750000   0.00740000
    1HEX    C13   15  -0.03330000   0.07420000  -0.04620000
    1HEX    C14   16  -0.16120000   0.00180000  -0.06790000
    1HEX   

UnsupportedCutoffMethodError: PME is not valid with a non-periodic system.

### Appendix: Validating the conversion to GROMACS and LAMMPS files

If GROMACS and/or LAMMPS are installed on your machine, the same comparisons can also take place with those engines. They are available via `conda` by running a command like:

```conda install gromacs lammps -c conda-forge -c bioconda```

In [ ]:
from distutils.spawn import find_executable
from pprint import pprint

from openff.interchange.drivers import get_gromacs_energies, get_lammps_energies

In [ ]:
if find_executable("lmp_serial"):
    pprint(get_lammps_energies(interchange).energies)

In [ ]:
if find_executable("gmx"):
    pprint(get_gromacs_energies(interchange).energies)

Finally, there is a helper function `get_summary_data` that will attempt to run drivers of each engine. A summary reported is prepared as a Pandas `DataFrame`.